# Predicting Bitcoin Prices

I'm trying to predict the price of bitcoin. I found a publicly available datasource on bitcoin prices, which I hope to use as a

In [1]:
from __future__ import absolute_import, division, print_function

import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import time

print(tf.__version__)

1.13.1


The first step is to obtain the data set: 

This took some searching, github to the rescue? Maybe. I'm looking for data by the minute, but perhaps daily values will do for now. 

In [2]:
# bitcoin_data_path = keras.utils.get_file(time.strftime('%Y-%m-%d_%H%M') + '_bitcoin.csv', 'https://github.com/Zombie-3000/Bitfinex-historical-data/blob/master/BTCUSD/Candles_1m/2013/merged.csv')
bitcoin_data_path = '/home/thwee/Downloads/BTC-USD (3).csv'
bitcoin_data_path

'/home/thwee/Downloads/BTC-USD (3).csv'

In [3]:
column_names = ['Date','Open','High','Low','Close',
                'Adj Close', 'Volume'] 

raw_dataset = pd.read_csv(bitcoin_data_path, names=column_names, sep=',')
raw_dataset


,Date,Open,High,Low,Close,Adj Close,Volume
0,Date,Open,High,Low,Close,Adj Close,Volume
1,2013-12-12,919.929993,940.979980,839.510010,899.979980,899.979980,8798283
2,2013-12-13,899.979980,989.890015,882.109985,936.770020,936.770020,16173632
3,2013-12-14,936.770020,947.929993,875.109985,908.940002,908.940002,10532106
4,2013-12-15,908.940002,927.900024,837.979980,919.880005,919.880005,13302074
5,2013-12-16,919.880005,924.940002,714.030029,759.919983,759.919983,32899151
6,2013-12-17,759.919983,779.960022,678.880005,714.969971,714.969971,22933905
7,2013-12-18,714.969971,716.989990,454.859985,540.979980,540.979980,62529030
8,2013-12-19,540.979980,745.960022,522.440002,731.950012,731.950012,29980041
9,2013-12-20,731.950012,774.390015,621.250000,649.960022,649.960022,24291048


The first seach I did for bitcoin prices turned up a [daily average](https://coinmarketcap.com/currencies/bitcoin/historical-data/).



In [4]:
raw_dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,Date,Open,High,Low,Close,Adj Close,Volume
1,2013-12-12,919.929993,940.979980,839.510010,899.979980,899.979980,8798283
2,2013-12-13,899.979980,989.890015,882.109985,936.770020,936.770020,16173632
3,2013-12-14,936.770020,947.929993,875.109985,908.940002,908.940002,10532106
4,2013-12-15,908.940002,927.900024,837.979980,919.880005,919.880005,13302074


In [5]:
dataset = raw_dataset.copy()
dataset.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1909,2019-03-04,3812.310059,3831.010010,3705.139893,3731.280029,3731.280029,147245676
1910,2019-03-05,3731.280029,3893.520020,3720.489990,3874.179932,3874.179932,165062256
1911,2019-03-06,3874.179932,3916.290039,3839.389893,3874.979980,3874.979980,129520549
1912,2019-03-07,3874.979980,3911.760010,3853.199951,3882.610107,3882.610107,146201726
1913,2019-03-08,3882.610107,3930.870117,3875.899902,3887.719971,3887.719971,48057488


In [6]:
dataset.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [8]:
import seaborn as sns
sns.set(style="ticks", color_codes=True)

In [9]:
sns.pairplot(raw_dataset)

IndexError: index -1 is out of bounds for axis 0 with size 0

<Figure size 0x0 with 0 Axes>

Oh well, time to move on. 

In [14]:
train_stats = train_dataset.describe()

train_stats.pop("Close")
train_stats = train_stats.transpose()

train_stats

,count,unique,top,freq
Date,1531,1528,2015-03-29,2
Open,1531,1522,237.410004,2
High,1531,1520,612.679993,2
Low,1531,1521,592.789978,2
Adj Close,1531,1521,288.369995,2
Volume,1531,1531,2034965,1
mean,NaN,NaN,NaN,NaN


In [15]:
train_labels = train_dataset.pop('Close')
test_labels = test_dataset.pop('Close')

In [16]:

# y = (x - min) / (max - min)

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

KeyError: 'mean'